# Lab 05: Working with Applications and Service Principals

This lab will guide you through managing applications and service principals in Entra ID using the Microsoft Graph PowerShell SDK. You'll learn how to register applications, configure permissions, create custom app roles, and manage ownership.

## Required Permissions

To complete this lab, you need the following Microsoft Graph API permissions:

- **Application.ReadWrite.All**: Required to create and manage application registrations
- **Directory.Read.All**: Required to read directory objects (users, groups)

These are privileged permissions that require administrative consent in your Entra ID tenant.

## Authentication with Microsoft Graph

Before you can perform any operations, you need to authenticate to Microsoft Graph using the `Connect-MgGraph` cmdlet. You'll need to sign in with an account that has appropriate administrative privileges.

For this lab, we'll use delegated authentication.

In [ ]:
# Connect to Microsoft Graph with the required permissions
Connect-MgGraph -Scopes "Application.ReadWrite.All", "Directory.Read.All"

# Verify the connection and permissions
Get-MgContext | Out-String

## Step 0: Specify your user name

You are all working in the same tenant. Edit the following code cell to specify your user name (for example `userone`, `usertwo`...).

Don't forget to run it. That user name will be used for naming resources during the labs.

In [ ]:
# Specify your user name (for example "userone", "usertwo"...)
# $your_username = '<USERNAME>'
$your_username = 'userone'

## Task 1: Query and Search for Applications

Before creating new applications, let's explore how to search for existing applications in your tenant. You'll learn different query techniques using `-Filter` and `-Search` parameters.

**Key Concepts:**
- `-Filter` with `startswith()` performs prefix matching
- `-Search` performs substring matching (requires `-ConsistencyLevel eventual`)
- Advanced queries with `$count` require `-ConsistencyLevel eventual`

In [ ]:
# Use -Filter to find applications with display names starting with 'Microsoft'
# Returns the first matching application, sorted by displayName
Get-MgApplication -Filter "startswith(displayName, 'Microsoft')" -CountVariable CountVar -Top 1 -Sort 'displayName' -ConsistencyLevel eventual | Format-Table DisplayName, AppId, SignInAudience | Out-String

Write-Host "Total applications found: $CountVar" -ForegroundColor Cyan

In [ ]:
# Use -Search to find applications containing 'Graph' in the display name
# -Search requires double quotes around the search expression
Get-MgApplication -Search '"displayName:Graph"' -CountVariable CountVar -Property 'appId,displayName,publisherDomain,signInAudience' -ConsistencyLevel eventual -Top 5 | Format-Table DisplayName, AppId, PublisherDomain | Out-String

Write-Host "Total applications found: $CountVar" -ForegroundColor Cyan

## Task 2: Register a New Application

Now you'll create a new application registration. The `displayName` is the only required property. When you create an application:

- It receives a globally unique ID (`id`) for management operations
- It receives an application ID (`appId`) for authentication flows
- Default `signInAudience` is 'AzureADMyOrg' (single tenant)
- You're automatically assigned as owner (with admin privileges)

In [ ]:
# Register a new application with minimal configuration
$appName = "Lab05-App-{0}" -f $your_username

$params = @{
    displayName = $appName
}

$app = New-MgApplication -BodyParameter $params

# Display the created application
$app | Format-List Id, AppId, DisplayName, PublisherDomain, SignInAudience | Out-String

# Store the application ID for later use
$applicationId = $app.Id
$appId = $app.AppId

### Understanding Application IDs

Notice that the application has two important IDs:

1. **id** (Object ID): Globally unique in the Entra ID ecosystem. Use this for management operations with Microsoft Graph API (e.g., `Update-MgApplication`, `Remove-MgApplication`)

2. **appId** (Application/Client ID): Use this for authentication flows and when looking up the corresponding service principal

**Important:** Store both IDs as we'll use them throughout the lab.

## Task 3: Configure Application Properties

Now let's configure additional properties for the application, including:
- **tags**: For categorization and visibility control
- **info**: Branding and legal information URLs
- **web**: Web application settings (redirect URIs, logout URL)
- **serviceManagementReference**: Contact information for service owners

In [ ]:
# Configure comprehensive application properties
$params = @{
    tags = @(
        'Lab05'           # Custom tag for categorization
        'Training'        # Custom tag for categorization
    )
    info = @{
        logoUrl = 'https://cdn.pixabay.com/photo/2016/03/21/23/25/link-1271843_1280.png'
        marketingUrl = 'https://www.contoso.com/app/marketing'
        privacyStatementUrl = 'https://www.contoso.com/app/privacy'
        supportUrl = 'https://www.contoso.com/app/support'
        termsOfServiceUrl = 'https://www.contoso.com/app/termsofservice'
    }
    web = @{
        homePageUrl = 'https://www.contoso.com/'
        logoutUrl = 'https://www.contoso.com/logout'
        redirectUris = @(
            'https://localhost'
            'https://localhost:5001'
        )
    }
    serviceManagementReference = "Owner: Lab Training {0}" -f $your_username
}

# Apply the configuration to the application
Update-MgApplication -ApplicationId $applicationId -BodyParameter $params

Write-Host "Application properties updated successfully!" -ForegroundColor Green

# Retrieve and display the updated application
Get-MgApplication -ApplicationId $applicationId | Format-List DisplayName, Tags, PublisherDomain, ServiceManagementReference | Out-String

## Task 4: Configure API Permissions

Now you'll assign API permissions to your application. This is a critical step for defining what your application can access.

**IMPORTANT NOTES:**
1. You must pass in BOTH existing and new permissions when updating
2. Assigning permissions does NOT automatically grant them - you still need admin consent
3. Permission types:
   - `type = 'Scope'`: Delegated permission (acts on behalf of signed-in user)
   - `type = 'Role'`: Application permission (app acts with its own identity)

We'll configure permissions for Microsoft Graph API.

In [ ]:
# First, let's look up Microsoft Graph's service principal to understand the resource
$graphSP = Get-MgServicePrincipal -Filter "appId eq '00000003-0000-0000-c000-000000000000'"

$graphSP | Format-List DisplayName, AppId, Id, ServicePrincipalType | Out-String

In [ ]:
# Configure Microsoft Graph API permissions
# User.Read.All (delegated) and Group.Read.All (application)
$params = @{
    requiredResourceAccess = @(
        @{
            resourceAppId = '00000003-0000-0000-c000-000000000000'  # Microsoft Graph
            resourceAccess = @(
                @{
                    id = 'a154be20-db9c-4678-8ab7-66f6cc099a59'    # User.Read.All (delegated)
                    type = 'Scope'                                   # Delegated permission
                },
                @{
                    id = '5b567255-7703-4780-807c-7be8301ae99b'    # Group.Read.All (application)
                    type = 'Role'                                    # Application permission
                }
            )
        }
    )
}

# Apply the permissions to the application
Update-MgApplication -ApplicationId $applicationId -BodyParameter $params

Write-Host "API permissions configured successfully!" -ForegroundColor Green
Write-Host "Note: You still need to grant admin consent for these permissions." -ForegroundColor Yellow

### Verify Configured Permissions

Let's retrieve the application and examine the permissions we just configured.

In [ ]:
# Retrieve the application and display configured permissions
$app = Get-MgApplication -ApplicationId $applicationId

Write-Host "Required Resource Access:" -ForegroundColor Cyan
$app.RequiredResourceAccess | ForEach-Object {
    Write-Host "  Resource App ID: $($_.ResourceAppId)" -ForegroundColor Yellow
    $_.ResourceAccess | ForEach-Object {
        Write-Host "    Permission ID: $($_.Id), Type: $($_.Type)"
    }
}

## Task 5: Create Custom App Roles

App roles allow you to define custom permissions for your application, enabling role-based access control (RBAC). These roles can be assigned to users, groups, or other applications.

**Key Properties:**
- **allowedMemberTypes**: Who can be assigned ('User', 'Application', or both)
- **value**: Role claim value that appears in tokens (use this in your app code)
- **id**: Unique GUID for the role

In [ ]:
# Create custom app roles for your application
$params = @{
    appRoles = @(
        @{
            allowedMemberTypes = @('User')
            description = 'Read survey data'
            displayName = 'Survey.Read'
            id = (New-Guid).Guid
            isEnabled = $true
            value = 'Survey.Read'
        },
        @{
            allowedMemberTypes = @('User', 'Application')
            description = 'Write and modify survey data'
            displayName = 'Survey.Write'
            id = (New-Guid).Guid
            isEnabled = $true
            value = 'Survey.Write'
        }
    )
}

# Apply the app roles to the application
Update-MgApplication -ApplicationId $applicationId -BodyParameter $params

Write-Host "Custom app roles created successfully!" -ForegroundColor Green

# Retrieve and display the app roles
$app = Get-MgApplication -ApplicationId $applicationId
$app.AppRoles | Format-Table DisplayName, Value, AllowedMemberTypes, IsEnabled | Out-String

## Task 6: Work with Service Principals

Every application registration can have a corresponding service principal (enterprise application) in the tenant. 

**IMPORTANT**: The service principal is NOT automatically created when you register an application. It's created when:
- Admin consent is granted for the application's permissions
- A user signs in to the application for the first time
- You manually create it using `New-MgServicePrincipal`

The service principal is the local representation of the application that controls:
- Who can access the application
- What permissions the application has been granted in the tenant
- Application assignments and settings

Let's check if a service principal exists, and if not, create one.

In [ ]:
# Check if service principal already exists
$servicePrincipal = Get-MgServicePrincipal -Filter "appId eq '$appId'"

if ($servicePrincipal) {
    Write-Host "Service principal already exists!" -ForegroundColor Green
    $servicePrincipal | Format-List DisplayName, Id, AppId, ServicePrincipalType | Out-String
    $spId = $servicePrincipal.Id
} else {
    Write-Host "Service principal not found. Creating it now..." -ForegroundColor Yellow
    
    # Create the service principal
    $servicePrincipal = New-MgServicePrincipal -AppId $appId
    
    Write-Host "Service principal created successfully!" -ForegroundColor Green
    $servicePrincipal | Format-List DisplayName, Id, AppId, ServicePrincipalType | Out-String
    $spId = $servicePrincipal.Id
}

## Task 7: Manage Application and Service Principal Owners

Managing ownership is critical for application lifecycle management. Best practices include:
- Assign at least 2 owners to critical applications
- Avoid orphaned applications (no owners)
- Regularly audit application ownership

Let's verify and manage ownership for your application.

In [ ]:
# Verify current owners of the application
Write-Host "Current Application Owners:" -ForegroundColor Cyan
Get-MgApplicationOwner -ApplicationId $applicationId | Select-Object -ExpandProperty AdditionalProperties | Out-String

In [ ]:
# Optional: Add another user as an owner
# Replace '<user-upn-or-id>' with an actual user's UPN or object ID
# Uncomment the lines below to add an owner

# $ownerUser = Get-MgUser -UserId '<user-upn-or-id>'
# $params = @{
#     '@odata.id' = "https://graph.microsoft.com/v1.0/directoryObjects/$($ownerUser.Id)"
# }
# New-MgApplicationOwnerByRef -ApplicationId $applicationId -BodyParameter $params
# Write-Host "Owner added successfully!" -ForegroundColor Green

## Challenge Task 1: Find Orphaned or Single-Owner Applications

Identify applications in your tenant that have zero or only one owner. These applications may need additional owners for better management and continuity.

**Hint:** Use `-Filter` with `owners/$count` and `-ConsistencyLevel eventual`

In [ ]:
# Find applications with 0 or 1 owner
Get-MgApplication -Filter "owners/`$count eq 0 or owners/`$count eq 1" -CountVariable CountVar -Property 'id,displayName' -ConsistencyLevel eventual -Top 10 | Format-Table DisplayName, Id | Out-String

Write-Host "Total applications with 0 or 1 owner: $CountVar" -ForegroundColor Yellow

## Challenge Task 2: Explore Service Principal Types

Explore the different types of service principals in your tenant. Service principal types include:
- **Application**: Standard enterprise applications
- **ManagedIdentity**: System or user-assigned managed identities
- **Legacy**: Older service principals that may need migration
- **SocialIdp**: Social identity providers (Facebook, Google, etc.)

In [ ]:
# List all unique service principal types in the tenant
Write-Host "Service Principal Types in Tenant:" -ForegroundColor Cyan
Get-MgServicePrincipal -Top 1000 | Select-Object ServicePrincipalType -Unique | Out-String

In [ ]:
# Find all managed identity service principals
Write-Host "Managed Identity Service Principals:" -ForegroundColor Cyan
Get-MgServicePrincipal -Filter "servicePrincipalType eq 'ManagedIdentity'" -Top 5 | Format-Table DisplayName, ServicePrincipalType, AppId | Out-String

## Challenge Task 3: Explore the Microsoft Entra Application Gallery

The Microsoft Entra application gallery contains thousands of pre-integrated SaaS applications. You can browse and filter these applications programmatically.

**Note:** Publisher and category filters are CASE-SENSITIVE. Use exact casing like 'Microsoft', 'Productivity', etc.

In [ ]:
# Browse gallery applications published by Microsoft
Get-MgApplicationTemplate -Filter "Publisher eq 'Microsoft'" -Top 10 | Format-Table DisplayName, Publisher, Categories | Out-String

## Cleanup: Remove the Application

When you're finished with the lab, you can remove the application you created. This will also remove the associated service principal.

**Warning:** This action cannot be undone!

In [ ]:
# Uncomment the line below to remove the application
# Remove-MgApplication -ApplicationId $applicationId -Confirm:$false
# Write-Host "Application removed successfully!" -ForegroundColor Green

## Conclusion

You have successfully completed the following tasks:

1. Queried and searched for applications using different filter techniques
2. Registered a new application in Entra ID
3. Configured application properties (tags, branding, web settings)
4. Assigned API permissions to the application
5. Created custom app roles for role-based access control
6. Located and examined the corresponding service principal
7. Managed application ownership
8. Explored service principal types and the application gallery

These skills are essential for managing application registrations and implementing secure authentication and authorization in your applications using Microsoft Graph PowerShell.

### Key Takeaways:

- **Application vs Service Principal**: The application is the global definition; the service principal is the tenant-specific instance
- **Two IDs**: Use `id` (object ID) for management operations, `appId` for authentication
- **Permissions**: Assigning permissions doesn't grant them - you still need admin consent
- **Ownership**: Always maintain at least 2 owners for critical applications
- **App Roles**: Define custom permissions for fine-grained authorization in your applications